### Tbot - Trade Bot

 Tbot is a Robot of trade developed to trade cryptourrencies or s&p500 stocks, based on https/json queries. With few adjustments it may help traders to improve their workflow. In this file we shall compile three steps followed at the "Algorithimic Trading Using Python" course created by the api developer and completed by the developer of this api. ("https://freecodecamp.org")
 
### Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this api. 

In [1]:
import numpy as np
import pandas as pd
import requests
import secrets
import xlsxwriter
import math
import hmac
import time
import datetime
import hashlib
import binascii

### API token, secret key and acess keys
API tokens (and other sensitive information) shall be stored in a secrets.py file that doesn't get pushed to your local Git repository. The AcessKey and SecretKey are provided by the server admin.

In [2]:
from secrets import SecretKey
from secrets import AccessKey
from novadax import RequestClient as NovaClient

nova_client = NovaClient(AccessKey, SecretKey)

### Importing tht List of currencies, making the API Call and the table Dataframe
List of crypto-currencies, get balance, and table creation

In [3]:
balance = nova_client.get_account_balance()
bdata = balance['data']

bcolumns = ['Currency', 'Balance']
bdataframe = pd.DataFrame(columns = bcolumns,)
bs = 0
bcr = {}

for bsymbol in bdata:
    pdis = bdata[bs]
    bis = pdis['balance']
    bic = pdis['currency']
    bcr[f"{bic}"] = f"{bis}"   
    bdataframe.append(
        pd.Series([
            bic,
            bis,
        ], index = bcolumns),
        ignore_index = True)
    bdataframe = bdataframe.append(
    pd.Series([
        bic,
        bis,
    ], index=bcolumns),
        ignore_index=True)
    bs = bs + 1  

bdataframe.sort_values('Balance', ascending = False, inplace = True)
bdataframe.reset_index(drop = True, inplace = True)
bdataframe

,Currency,Balance
0,BTT,96.09424
1,USDT,0.09682579
2,ETH,0.0438716847
3,DGB,0.0353
4,LINK,0.0088601
5,DOGE,0.00881
6,XLM,0.00447
7,NULS,0.0044
8,IOTA,0.003905
9,XRP,0.0036


### Dolar price, timestamp and symbols 
Important cell defining symbols and the formula to the dolar price in terms of brazilian real(BRL), wich can be easily translated to other currencies or symbols.

In [4]:
symbol = 'ETH_USDT'
symbols = ['ETH_USDT', 'ETH_BRL']
currency = 'R$'
dsymbol = 'ETH_BRL'
ldolar = nova_client.list_trades(symbol, limit = 5)
my_columns = ['Symbol', 'Amount', 'Price', 'Number of Shares to Buy']
ldr_dataframe = pd.DataFrame(columns = my_columns)

for symbol in symbols:
    f = 0
    for i in ldolar['data']:
        ldolar = nova_client.list_trades(symbol, limit=10)
        ldar = ldolar['data']
        ldr = ldar[f]
        amount = ldr['amount']
        price = float(ldr['price'])
        cprice = float(amount) * price
        edolar = bcr['USDT']
        eth_wal = bcr['ETH']
        
        cwal = float(eth_wal) * price + float(edolar)
        
        sharestobuy = float(cwal/price)
                            
        
        baseVolume24h =  'N/A'
        tsp = (ldr['timestamp'])
        f = f + 1
        ldr_dataframe.append(
                    pd.Series(
                    [
                        symbol,
                        amount,
                        price,
                        sharestobuy,
                    ]
                    ,index = my_columns),
                ignore_index=True)

    
        ldr_dataframe = ldr_dataframe.append(
                pd.Series([
                        symbol,
                        amount,
                        price,
                        sharestobuy,
                    
                      ]
                ,index = my_columns), 
                ignore_index = True)

ldr_dataframe.sort_values('Price', ascending = True, inplace = True)
ldr_dataframe.reset_index(drop = True, inplace = True)
ldr_dataframe

,Symbol,Amount,Price,Number of Shares to Buy
0,ETH_USDT,0.0711,1546.64,0.043934
1,ETH_USDT,0.0352,1564.99,0.043934
2,ETH_USDT,2.4176,1565.80,0.043934
3,ETH_USDT,0.0068,1566.10,0.043934
4,ETH_USDT,0.0235,1569.73,0.043933
5,ETH_BRL,2.4175824,9100.00,0.043882
6,ETH_BRL,0.03258585,9194.92,0.043882
7,ETH_BRL,0.09775762,9212.11,0.043882
8,ETH_BRL,0.12084882,9226.86,0.043882
9,ETH_BRL,1.17441654,9239.09,0.043882


### Symbol position
For the execution to be done independently for each symbol, the below cell is needed:

In [5]:
my_columns = ['Symbol', 'Currency Price', 'Number of Shares to Buy']
e_dataframe = pd.DataFrame(columns = my_columns)
for symbol in symbols:
    r = 0
    for i in ldolar['data']:  
        fddata = nova_client.list_trades(symbol, limit=10)
        fdata = fddata['data']
        fdt = fdata[r]
        price = fdt['price']
        amount = fdt['amount']
        cprice = float(amount) * float(price)
        edolar = bcr['USDT']
        eth_wal = float(bcr['ETH']) * float(price)
        edolarwal = float(bcr['USDT']) + float(eth_wal)
        shares_to_buy = (float(edolarwal)/float(price))
        etsp = (fdt['timestamp'])
        for i in fddata['data'][:1]:
            e_dataframe.append(
                pd.Series(
                    [
                        symbol,
                        price,
                        shares_to_buy
                    ]
                    ,index = my_columns),
                ignore_index=True)
    
    
            e_dataframe = e_dataframe.append(
                                                pd.Series([
                                                    symbol, 
                                                    price,
                                                    shares_to_buy
                                                    ],
                                                    index = my_columns
                                                    ),
                                                ignore_index = True)

e_dataframe

,Symbol,Currency Price,Number of Shares to Buy
0,ETH_USDT,1566.1,0.043934
1,ETH_USDT,1566.1,0.043934
2,ETH_USDT,1566.1,0.043934
3,ETH_USDT,1566.1,0.043934
4,ETH_USDT,1566.1,0.043934
5,ETH_USDT,1566.1,0.043934
6,ETH_USDT,1566.1,0.043934
7,ETH_USDT,1566.1,0.043934
8,ETH_USDT,1566.1,0.043934
9,ETH_USDT,1566.1,0.043934


### Sending the requisition to the web
The content in the order requisition should be sent to the server following the instructions given by the company or individual that holds the coin. Here as columns the last used titles shall be all used in one go to give a more complete dataframe in our xlsx database.

In [24]:
#20210222#v0-9#import all the list itens from the last dataframes and send them to the xlsx file. 
#orders
listtrades = nova_client.list_orders(symbol, 'FILLED', limit=10)
ldata = listtrades['data']
lindice= ldata[0]
amount = lindice['amount']
my_columns = ['Symbol', 'Ammount','Price', 'Number of Shares to Buy', 'momentum-raw-data']
f_dataframe = pd.DataFrame(columns = my_columns)

for symbol in symbols:
    listtrades = nova_client.list_trades(symbol, limit=5)
    candlestick = nova_client.get_kline(symbol, unit = 'ONE_WEE', from_score='161405418348', to_score='1614094183748')
    f = 0
    for i in listtrades['data']:
        ldata = listtrades['data']
        ldc = ldata[f]
        amount = ldc['amount']
        price = float(ldc['price'])
        
        amount = ldc['amount']
        cprice = float(amount) * float(price)
        edolar = bcr['USDT']
        eth_wal = float(bcr['ETH']) * float(price)
        edolarwal = float(bcr['USDT']) + float(eth_wal)
        sharestobuy = (float(edolarwal)/float(price)) 
              
        candledata = candlestick['data']
        ltsp = (ldc['timestamp'])
        f_dataframe.append(
                    pd.Series(
                    [
                        symbol,
                        amount,
                        price,
                        sharestobuy,
                        candledata,
                    ]
                    ,index = my_columns),
                ignore_index=True)

    
        f_dataframe = f_dataframe.append(
                pd.Series([
                        symbol,
                        amount,
                        price,
                        sharestobuy,
                        candledata,
                    
                      ]
                ,index = my_columns), 
                ignore_index = True)

f_dataframe.sort_values('Price', ascending = True, inplace = True)
f_dataframe.reset_index(drop = True, inplace = True)
f_dataframe

,Symbol,Ammount,Price,Number of Shares to Buy,momentum-raw-data
0,ETH_USDT,0.0528,1596.10,0.043932,[]
1,ETH_USDT,0.0528,1596.10,0.043932,[]
2,ETH_USDT,0.0528,1596.10,0.043932,[]
3,ETH_USDT,0.0528,1596.10,0.043932,[]
4,ETH_USDT,0.0528,1596.10,0.043932,[]
5,ETH_BRL,0.01163213,8979.23,0.043882,[]
6,ETH_BRL,0.01163213,8979.23,0.043882,[]
7,ETH_BRL,0.01163213,8979.23,0.043882,[]
8,ETH_BRL,0.01163213,8979.23,0.043882,[]
9,ETH_BRL,0.01163213,8979.23,0.043882,[]


In [23]:
candle = nova_client.get_kline('ETH_USDT', unit ='ONE_WEE',from_score='1567619280', to_score='1567620000')
print(candle, ltsp)

{'code': 'A10000', 'data': [], 'message': 'Success'} 1614094450410


In [8]:
#sort price [ok]
#wallet_amount [ok]
#buy lower(wallet_amount - amount)     
#sell + profit(constant*market momentum)
#if sell = true; [buy lower < sell][(sell - (contant*momentum)/2 or (buy + constant/momentum)/2
#P = S - B; SUM(Pi to Pn)  = Pi{(S - D) or (S - B)}
#in the if statement one more contidional is needed at the "or" statment. If or else(momentum>=x or else)

### Format and export xlsx file
In the last section of the tbot a xlsx file is saved with the order table for later reading purposes.

In [9]:
writer = pd.ExcelWriter('orders.xlsx', engine = 'xlsxwriter')
f_dataframe.to_excel(writer, sheet_name = 'orderlist', index = False)

In [10]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [11]:
column_formats = {
    'A':['Symbol', string_template],
    'B':['Amount', float_template],
    'C':['Price', dollar_template],
}

for column in column_formats.keys():
    writer.sheets['orderlist'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['orderlist'].write(f'{column}1', column_formats[column][0], column_formats[column][1])
   

In [12]:
writer.save()